### Imports

In [1]:
import os
import shutil
from converter import convert_all
from data.get_data import get_data
from GAN import run_GAN

#### GAN training

In [5]:
%%capture
run_GAN(trained=False) # train GAN

Found 2 files belonging to 1 classes.
Epoch 1/1000


KeyError: 'The optimizer cannot recognize variable conv2d_12/kernel:0. This usually means you are trying to call the optimizer to update different parts of the model separately. Please call `optimizer.build(variables)` with the full list of trainable variables before the training loop or use legacy optimizer `tf.keras.optimizers.legacy.Adamax.'

### Using GAN to generate images

In [ ]:
run_GAN(trained=True) # generate images using pretrained GAN model

### Downloading images

In [ ]:
get_data(per_page=10, image_type="sunset") # download images for training

##### Move downloaded images to "to_convert" directory from which they can be processed by the next cell

In [6]:
source_directory = os.path.join("data", "downloaded")
destination_directory = os.path.join("data", "to_convert")
files = os.listdir(source_directory)

for file in files:
    source_path = os.path.join(source_directory, file)
    destination_path = os.path.join(destination_directory, file)
    shutil.move(source_path, destination_path)

### Converting images

In [7]:
convert_all(block_size=4) # prepare downloaded images to fit GAN model